In [1]:
import datetime as dt

date_s = dt.datetime(2024, 11, 16, 0, 0, 0, tzinfo=dt.UTC)
date_e = dt.datetime(2024, 11, 17, 0, 0, 0, tzinfo=dt.UTC)

In [2]:
from urllib import parse

base_url = "https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py?"
params = {
    "sts": date_s.isoformat().replace("+00:00", "Z"),
    "ets": date_e.isoformat().replace("+00:00", "Z"),
    "artcc": "_ALL",
    "fmt": "csv",
}
url = base_url + parse.urlencode(params)
url

'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py?sts=2024-11-16T00%3A00%3A00Z&ets=2024-11-17T00%3A00%3A00Z&artcc=_ALL&fmt=csv'

In [3]:
import pandas as pd

df = pd.read_csv(url)

In [4]:
df = df[["VALID", "REPORT"]]
df = df.rename(columns={"VALID": "Timestamp", "REPORT": "Report"})
df["Timestamp"] = pd.to_datetime(df["Timestamp"], format=f"%Y%m%d%H%M")
df

,Timestamp,Report
0,2024-11-16 00:00:00,SAW UA /OV KSAW/TM 0000/FL036/TP B190/SK OVC03...
1,2024-11-16 00:00:00,CRG UA /OV CRG/TM 0000/FL370/TP B738/TB NEG
2,2024-11-16 00:02:00,MTJ UA /OV MTJ045010/TM 0002/FL180/TP EA50/TB ...
3,2024-11-16 00:02:00,PGA UA /OV TBC080040/TM 0002/FL200/TP PC12/TB ...
4,2024-11-16 00:03:00,ORD UA /OV CGT330020/TM 0003/FL060/TP B738/SK ...
...,...,...
2466,2024-11-16 23:57:00,PLU UA /OV SEA160016 /TM 2357 /FL040 /TP C172 ...
2467,2024-11-16 23:58:00,UA /OV 5430N11900W /TM 2358 /FL310 /TP B77L /T...
2468,2024-11-16 23:58:00,MKK UA /OV BAMBO/TM 2358/FL130/TP A333/SK TOPS...
2469,2024-11-16 23:59:00,CID UA /OV CID/TM 2359/FL300/TP B738/TB CONS L...


In [5]:
from defs.report import PilotReport

def parse_pirep(row):
    try:
        report = PilotReport.parse(row["Report"], timestamp=row["Timestamp"])
        row["Priority"] = report.priority
        row["Location"] = report.location
        row["Altitude"] = report.altitude
        row["Aircraft"] = report.aircraft
        row["Turbulence"] = report.turbulence
        return row
    except:
        return row

In [6]:
transformed = df.apply(parse_pirep, axis=1)
transformed

,Aircraft,Altitude,Location,Priority,Report,Timestamp,Turbulence
0,M,err=None min=3600 max=3600,lat=0.0 lon=0.0,UA,SAW UA /OV KSAW/TM 0000/FL036/TP B190/SK OVC03...,2024-11-16 00:00:00,[]
1,M,err=None min=37000 max=37000,lat=0.0 lon=0.0,UA,CRG UA /OV CRG/TM 0000/FL370/TP B738/TB NEG,2024-11-16 00:00:00,[duration=None intensity=<Intensity.NEG: 'NEG'...
2,L,err=None min=18000 max=18000,lat=0.0 lon=0.0,UA,MTJ UA /OV MTJ045010/TM 0002/FL180/TP EA50/TB ...,2024-11-16 00:02:00,[duration=None intensity=<Intensity.MOD: 'MOD'...
3,L,err=None min=20000 max=20000,lat=0.0 lon=0.0,UA,PGA UA /OV TBC080040/TM 0002/FL200/TP PC12/TB ...,2024-11-16 00:02:00,[duration=None intensity=<Intensity.NEG: 'NEG'...
4,M,err=None min=6000 max=6000,lat=0.0 lon=0.0,UA,ORD UA /OV CGT330020/TM 0003/FL060/TP B738/SK ...,2024-11-16 00:03:00,[]
...,...,...,...,...,...,...,...
2466,L,err=None min=4000 max=4000,lat=0.0 lon=0.0,UA,PLU UA /OV SEA160016 /TM 2357 /FL040 /TP C172 ...,2024-11-16 23:57:00,[]
2467,H,err=None min=31000 max=31000,lat=54.5 lon=119.0,UA,UA /OV 5430N11900W /TM 2358 /FL310 /TP B77L /T...,2024-11-16 23:58:00,[duration=None intensity=<Intensity.MOD: 'MOD'...
2468,H,err=None min=13000 max=13000,lat=0.0 lon=0.0,UA,MKK UA /OV BAMBO/TM 2358/FL130/TP A333/SK TOPS...,2024-11-16 23:58:00,[]
2469,M,err=None min=30000 max=30000,lat=0.0 lon=0.0,UA,CID UA /OV CID/TM 2359/FL300/TP B738/TB CONS L...,2024-11-16 23:59:00,[duration=<Duration.CON: 'CONS'> intensity=<In...
